In [11]:
import networkx as nx 
from glob import glob
import pandas as pd 
import pickle as pkl
import csv
from tqdm import tqdm
import bz2
import json
import requests as rq
import time
import lzma

In [12]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='Parsing.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [13]:
dump = "/10TBdrive/datashare/Reddit/submissions/RS_2017-10.bz2"

In [14]:
openedDump = bz2.BZ2File(dump, "r")

In [15]:
submissions_bz = glob("/10TBdrive/datashare/Reddit/submissions/*.bz2")
submissions_xz = glob("/10TBdrive/datashare/Reddit/submissions/*.xz")

submissions = submissions_bz + submissions_xz

In [16]:
# submissions

In [17]:
lines = 0
for line in openedDump:
    if lines > 2:
        break
    lineData = json.loads(line.strip())
    print(lineData)
    lines+=1

{'archived': False, 'author': 'SealTeamSith', 'author_flair_css_class': 'red', 'author_flair_text': 'Transgender', 'brand_safe': False, 'contest_mode': False, 'created_utc': 1506816000, 'distinguished': None, 'domain': 'self.RightwingLGBT', 'edited': False, 'gilded': 0, 'hidden': False, 'hide_score': False, 'id': '73ig7j', 'is_crosspostable': False, 'is_reddit_media_domain': False, 'is_self': True, 'is_video': False, 'link_flair_css_class': 'rainbow', 'link_flair_text': 'GO VOTE!', 'locked': False, 'media': None, 'media_embed': {}, 'num_comments': 20, 'num_crossposts': 0, 'over_18': False, 'parent_whitelist_status': None, 'permalink': '/r/RightwingLGBT/comments/73ig7j/im_going_to_be_applying_some_css_changes_this/', 'pinned': False, 'retrieved_on': 1510382444, 'score': 7, 'secure_media': None, 'secure_media_embed': {}, 'selftext': '', 'spoiler': False, 'stickied': False, 'subreddit': 'RightwingLGBT', 'subreddit_id': 't5_3dc8i', 'suggested_sort': None, 'thumbnail': 'self', 'thumbnail_he

In [18]:
def getSubRedditMapping(filePaths , limit=None , comments_thresh=10):
    import json
    import bz2
    import lzma
    returnData = dict()
    file_count = 0
    total = len(filePaths)
    for path in tqdm(filePaths):
        extension = path.split('.')[-1]
        if extension == 'bz2':
            openedDump = bz2.BZ2File(path, "r")
        else:
            openedDump = lzma.open(path)
        count = 0
        print("=============Parsing============")
        logging.debug("=============Parsing============")
        st = time.time()
        for line in openedDump:
            try:
                lineData = json.loads(line.strip())
                if lineData['num_comments'] >= comments_thresh:
                    sub_id = lineData['id']
                    subreddit = lineData['subreddit']
                    subreddit_id = lineData['subreddit_id']
                    returnData[sub_id] = {}
                    returnData[sub_id]['subName'] = subreddit
                    returnData[sub_id]['subId'] = subreddit_id
            except:
                logging.warn(f"Incomplete record found")
            if count%1000000 == 0:
                end = time.time()
                delta = end-st
                print("Done parsing %d posts in %d seconds",(count,delta))
                
                logging.debug(f"Done parsing {count} posts in {delta} seconds")
                st = time.time()
            count+=1
        openedDump.close()
        
        logging.debug(f"Done {file_count} out of  {total} Files ")
        file_count+=1
    return returnData

In [ ]:
graph_to_sub_map = getSubRedditMapping(submissions)

  0%|          | 0/109 [00:00<?, ?it/s]

=============Parsing============
Done parsing %d posts in %d seconds (0, 0.035041093826293945)
Done parsing %d posts in %d seconds (1000000, 63.67950677871704)


  1%|          | 1/109 [01:08<2:04:09, 68.97s/it]

=============Parsing============
Done parsing %d posts in %d seconds (0, 0.014872550964355469)
Done parsing %d posts in %d seconds (1000000, 65.85064244270325)


  2%|▏         | 2/109 [03:18<2:35:30, 87.20s/it]

...

=============Parsing============
Done parsing %d posts in %d seconds (0, 0.015614748001098633)
Done parsing %d posts in %d seconds (1000000, 70.96942186355591)


 11%|█         | 12/109 [1:11:55<8:02:50, 298.66s/it] 

=============Parsing============
Done parsing %d posts in %d seconds (0, 0.016551971435546875)
Done parsing %d posts in %d seconds (1000000, 74.20077443122864)
Done parsing %d posts in %d seconds (2000000, 70.68603205680847)


 12%|█▏        | 13/109 [1:14:47<6:56:59, 260.61s/it]

=============Parsing============
Done parsing %d posts in %d seconds (0, 0.03762054443359375)
Done parsing %d posts in %d seconds (1000000, 72.8566620349884)


In [ ]:
%store graph_to_sub_map

In [ ]:
# graph_to_sub_map = {}
# for s in submissions:
#     sub = pkl.load(open(s,'rb'))
#     for k in sub:
#         subreddit = sub[k].split('/')[2] j
#         graph_to_sub_map[k] = subreddit


In [ ]:
# {k: graph_to_sub_map[k] for k in list(graph_to_sub_map)[:2]}

In [ ]:
graphs = glob('/10TBdrive/datashare/Reddit/graphs/*.pkl')

In [ ]:
len(graphs)

In [ ]:
g = pkl.load(open(graphs[10],'rb'))

In [ ]:
graphs[10]

In [ ]:
g.nodes(data=True)

In [ ]:
# g.edges()

In [ ]:
with open('/10TBdrive/datashare/Reddit/AllParsedData_2005_to_2012_and_2017.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, ['author','downs','ups','text','time','src_id','dest_id','dest_author','depth','subreddit'],delimiter='|')
    w.writeheader()
    for gra in tqdm(graphs):
        g = pkl.load(open(gra,'rb'))
        gid = gra.split('/')[-1].split('.')[0]
        try:
            subreddit = graph_to_sub_map[gid]
            nodes = g.nodes(data=True)
            node_dict = {n[0] : n[1]  for n in nodes}
            for u , v in g.edges():
                if node_dict[u]['author']!='[deleted]' and node_dict[v]['author']!='[deleted]' :
                    row= {'author': node_dict[u]['author'] , 'downs' : node_dict[u]['downs'] , 'ups': node_dict[u]['ups'] , 'text':node_dict[u]['text'] , 'time':node_dict[u]['time'] , 'src_id' : u , 'dest_id' : v , 'dest_author' : node_dict[v]['author'] ,'depth':node_dict[u]['depth'],'subreddit':subreddit }
                    w.writerow(row)
                else:
                    continue
        except:
            print("Some graph is malformed!! ")
            logging.debug("Some graph is malformed!!")

In [ ]:
# reddit_data = pd.read_csv('/10TBdrive/datashare/Reddit/AllParsedData_2017_V3.csv',sep='|')

In [ ]:
# len(reddit_data)

In [ ]:
# reddit_data.head()

In [ ]:
# len(reddit_data['author'].unique())